In [ ]:
!nvidia-smi

Tue Feb  8 14:07:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import os
import numpy as np

from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openpyxl==3.0.0

     |████████████████████████████████| 172 kB 6.3 MB/s 
  Created wheel for openpyxl: filename=openpyxl-3.0.0-py2.py3-none-any.whl size=241207 sha256=62e4fd77f76cc7f1cf2d80ff00c90589e9c7fd9489cfc3cb8f9f73b1b99c609b
  Stored in directory: /root/.cache/pip/wheels/c7/64/ff/ce98f6e1d2701ae8e216c875da62feed2839ac8a3cae0ab8af
Successfully built openpyxl
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9


In [ ]:
# загружаем набор строк 
esm = pd.read_excel('/content/коды ЕСМ.xlsx')
df_esm = esm.dropna(subset=[ 'Наименование ru', 'Полное наименование ru'])

In [ ]:
df_esm.head()

,Код,Наименование ru,Полное наименование ru,Нормативный документ,Единица измерения базовая,Вес брутто,Вес нетто,Обезличенная позиция,Класс,Статус материала ЕСМ,Блокировка,Блокировка на уровне завода,Эталонная запись
0,1240638,Кресло Реплай 466160MT,"Кресло Реплай, на колесиках 466160MT",КАТАЛОГ,ШТ,1,1,Нет,G4870,3.0,Нет,Нет,NaN
1,1239944,Стенд: Действия насел в ч/с техног.харак,Стенд: Действия населения в черезвычайных ситу...,ТУ ПРОИЗВОДИТЕЛЯ,ШТ,1,1,Нет,G4870,3.0,Да,Нет,NaN
2,1239962,Стенд: Первая медицинская помощь,"Стенд: Первая медицинская помощь. Размер: 1,0Х...",ТУ ПРОИЗВОДИТЕЛЯ,ШТ,1,1,Нет,G4870,3.0,Да,Нет,NaN
3,1239963,Стенд: Первая медицинская помощь при ЧС,Стенд: Первая медицинмкая помощь при ЧС. Разме...,ТУ ПРОИЗВОДИТЕЛЯ,ШТ,1,1,Нет,G4870,3.0,Да,Нет,NaN
4,1239964,Стенд: Приборы радиацион. и хим.разведки,Стенд: Приборы радиационной и химической разве...,ТУ ПРОИЗВОДИТЕЛЯ,ШТ,1,1,Нет,G4870,3.0,Да,Нет,NaN


In [ ]:
# загружаем набор строк справочника 
df_ita = pd.read_excel("/content/массив.xlsx")
df_ita = df_ita.dropna(subset=[ 'Наименование'])

In [ ]:
df_ita.head()

,Созданный код в ЕСМ,Код номенклатуры,Наименование
0,NaN,000001070157,Кофемашина Gemani 220
1,NaN,000001070273,ШРЕДЕР ЕВА 1123С
2,NaN,000001099633,Автоматическая установка пожарной сигнализации...
3,NaN,000001151839,холодильник Indesit В16Т
4,NaN,000001156618,Брошюратор Fellowes/FS-562070


# Векторизация

Основные данные содержатся в текстовом формате. Необходимо уметь представлять их в виде векторов, для того, чтобы с ними было возможно работать.

В данном примере для этого используется модель [Universal Sentence Encoder](https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3)


In [ ]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 6.8 MB/s 


In [ ]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import tensorflow as tf

In [ ]:
# загружаем модель USE
nlp = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [ ]:
# определяем функцию, которая будет переводить набор текстов в набор векторов
def vectorize_texts(texts):
    vecs = nlp(texts)
    return vecs

In [ ]:
# (пример) из набора из 4 текстов получаем матрицу 4x512 (4 вектора размерности 512)
vectors = vectorize_texts(["Привет", "Здравствуй", "Добрый день!", "Кошка"])
vectors.shape

TensorShape([4, 512])

# Сравнение векторов

Для того, чтобы найти схожесть между двумя текстовыми документами, достаточно найти близость их векторных представлений.

Для этого используем [косинусное сходство](https://en.wikipedia.org/wiki/Cosine_similarity)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# все тексты примерно похожи друг на друга, кроме "Кошка"
cosine_similarity(vectors, vectors)

array([[1.0000001 , 0.9483943 , 0.8001865 , 0.29887924],
       [0.9483943 , 1.        , 0.80816734, 0.26898402],
       [0.8001865 , 0.80816734, 1.        , 0.14897767],
       [0.29887924, 0.26898402, 0.14897767, 1.0000001 ]], dtype=float32)

# Предобработка текста

В этом примере в качестве обоработки лишь удаляется пунктуация и нормализуются пробелы.

Другие предобработки, которые можно попробовать:
* Удаление часто встречающихся слов ИЛИ использование их как отдельные признаки ("Наименование", "Оборудование" ...)
* Расшифровка аббревиатур
* Выделение единиц меры в отдельный признак (млн руб., тонн, см, м, ...)


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

stopwords = stopwords.words("russian")
stemmer = SnowballStemmer("russian")

def preprocess_text(text):
    text = re.sub(r"<.*?>", "", text)
    text = text.lower()

    # пунктуация и пробелы
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub('[\s]+', ' ', text)

    tokens = text.split(" ")

    # стоп-слова и стемминг
#     no_stop_lemm_tokens = [stemmer.stem(token) for token in tokens if token not in stopwords]

    return " ".join(tokens)

# Оценка схожести

Имея возможность векторизовать текст и сравнить схожесть векторных представлений, мы можем оценить сходство любой пары текстовых документов.

In [ ]:
# нахождение матрицы схожести между наборами векторов
def sim(vecs1, vecs2):
    return cosine_similarity(
        np.array(list(vecs1)).squeeze(), 
        np.array(list(vecs2)).squeeze()
    )

In [ ]:
# нахождение матрицы схожести между наборами текстов
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def get_similarities(texts1, texts2, bs=32):
    b_t1 = list(batch(texts1, bs))
    b_t2 = list(batch(texts2, bs))
    
    vectors1 = []
    for texts in tqdm(b_t1):
        vectors1 += list(vectorize_texts(texts))

    result = {'index': [], 'similarity': []}
    for texts in tqdm(b_t2):
      sims = sim(vectors1, vectorize_texts(texts))
      result['index'] += list(np.argmax(sims, axis=0))
      result['similarity'] += list(np.amax(sims, axis=0))


    return result
    

In [ ]:
df_esm_min = df_esm.sample(1000)
df_ita_min = df_ita.sample(1000)

In [ ]:
# получаем матрицу сходства наименований
name_sims = get_similarities(
        df_esm["Наименование ru"].apply(preprocess_text), 
        df_ita["Наименование"].apply(preprocess_text)
    )

  0%|          | 0/3889 [00:00<?, ?it/s]

  0%|          | 0/117 [00:00<?, ?it/s]

In [ ]:
name_sims = pd.DataFrame(name_sims)

In [ ]:
df_res = pd.concat([df_ita.reset_index(), df_esm.iloc[list(name_sims['index'])].reset_index(), name_sims.reset_index()], axis=1).drop("index", 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
df_res2 = df_res[['Код', 'Наименование ru', 'Код номенклатуры', 'Наименование', 'similarity']]

In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 8.2 MB/s 


In [ ]:
df_res2.to_excel('result.xlsx', engine='xlsxwriter')

In [ ]:
!cp result.xlsx /content/drive/MyDrive